<a href="https://colab.research.google.com/github/ananyaarya02/Echoes-of-Emotion/blob/main/Echoes_of_Emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install tensorflow librosa transformers torch tensorflow tensorflow-hub




In [30]:
import kagglehub
import pandas as pd
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
import tensorflow_hub as hub
from transformers import pipeline
import torch










In [31]:
# Load Hugging Face token from Drive
with open('/content/drive/My Drive/huggingface_token.txt', 'r') as f:
    hf_token = f.read().strip()

# Authenticate with Hugging Face
from huggingface_hub import login
login(hf_token)


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
path = kagglehub.dataset_download("imsparsh/audio-speech-sentiment")


In [34]:
print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/imsparsh/audio-speech-sentiment/versions/2


In [35]:

path = "/root/.cache/kagglehub/datasets/imsparsh/audio-speech-sentiment/versions/2"

subfolders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

print("Subfolders:", subfolders)
print("Files:", files)


Subfolders: ['TRAIN', 'test_images', 'TEST', 'train_images']
Files: ['TRAIN.csv']


In [36]:
data = pd.read_csv('/root/.cache/kagglehub/datasets/imsparsh/audio-speech-sentiment/versions/2/TRAIN.csv')
print(data.head())


  Filename     Class
0  346.wav  Negative
1  163.wav   Neutral
2  288.wav  Negative
3  279.wav  Negative
4  244.wav  Negative


In [37]:
#to check if there are any missing files in the directory
train_files = os.listdir('/root/.cache/kagglehub/datasets/imsparsh/audio-speech-sentiment/versions/2/TRAIN/')
missing_files = [file for file in data['Filename'] if file not in train_files]
print("Missing Files:", missing_files)


Missing Files: []


In [38]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
    return np.hstack([mfccs, chroma])

In [39]:

train_dir = '/root/.cache/kagglehub/datasets/imsparsh/audio-speech-sentiment/versions/2/TRAIN/'
X_train = []
y_train = []
for index, row in data.iterrows():
    file_path = os.path.join(train_dir, row['Filename'])
    features = extract_features(file_path)
    X_train.append(features)
    y_train.append(row['Class'])

X_train = np.array(X_train)
y_train = np.array(y_train)



In [40]:
#encoding the sentiments

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
print(encoder.classes_)


['Negative' 'Neutral' 'Positive']


In [41]:
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train_encoded, test_size=0.2, random_state=42)


In [42]:
print(X_train.shape)


(250, 25)


In [43]:
rnn_model = Sequential([
    SimpleRNN(64, activation='relu', input_shape=(25, 1)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(len(set(y_train_encoded)), activation='softmax')
])

rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
rnn_model.fit(X_train_split, y_train_split, validation_data=(X_val, y_val), epochs=10, batch_size=32)
rnn_accuracy = rnn_model.evaluate(X_val, y_val, verbose=0)[1]
print(f"RNN Validation Accuracy: {rnn_accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 163ms/step - accuracy: 0.3771 - loss: 1.0950 - val_accuracy: 0.3000 - val_loss: 1.1057
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.3770 - loss: 1.0887 - val_accuracy: 0.3600 - val_loss: 1.1025
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3520 - loss: 1.0847 - val_accuracy: 0.3000 - val_loss: 1.0991
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.3693 - loss: 1.0760 - val_accuracy: 0.3000 - val_loss: 1.0838
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.3524 - loss: 1.0532 - val_accuracy: 0.2600 - val_loss: 1.0449
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4000 - loss: 1.0225 - val_accuracy: 0.5600 - val_loss: 0.9494
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5167 - loss: 0.9165 - val_accuracy: 0.6000 - val_loss: 0.7878
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5541 - loss: 0.7985 - val_accuracy: 0.6600 - val_loss: 0.6264

In [44]:
# CNN model
cnn_model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(25, 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(set(y_train_encoded)), activation='softmax')
])
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train_split, y_train_split, validation_data=(X_val, y_val), epochs=10, batch_size=32)
cnn_accuracy = cnn_model.evaluate(X_val, y_val, verbose=0)[1]
print(f"CNN Validation Accuracy: {cnn_accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.3953 - loss: 4.9807 - val_accuracy: 0.6400 - val_loss: 0.5529
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5838 - loss: 1.7866 - val_accuracy: 0.7200 - val_loss: 0.5046
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6750 - loss: 0.8182 - val_accuracy: 0.7600 - val_loss: 0.4237
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.6645 - loss: 0.6315 - val_accuracy: 0.7800 - val_loss: 0.3612
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.8573 - loss: 0.3354 - val_accuracy: 0.8600 - val_loss: 0.3106
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8739 - loss: 0.3492 - val_accuracy: 0.9000 - val_loss: 0.2616
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8878 - loss: 0.2735 - val_accuracy: 0.9400 - val_loss: 0.2004
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.9032 - loss: 0.2289 - val_accuracy: 0.9200 - val_loss: 0.1966


In [45]:
#Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_split, y_train_split)

accuracy = clf.score(X_val, y_val)
print(f"Validation Accuracy: {accuracy:.2f}")



Validation Accuracy: 0.98


In [46]:
import xgboost as xgb

xg_clf = xgb.XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
xg_clf.fit(X_train_split, y_train_split)
accuracy = xg_clf.score(X_val, y_val)
print(f"Validation Accuracy: {accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [11:15:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Validation Accuracy: 0.94


In [47]:
import lightgbm as lgb
# LightGBM classifier
lgb_clf = lgb.LGBMClassifier(n_estimators=100, random_state=42)
lgb_clf.fit(X_train_split, y_train_split, eval_set=[(X_val, y_val)])
accuracy = lgb_clf.score(X_val, y_val)
print(f"Validation Accuracy: {accuracy:.2f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1702
[LightGBM] [Info] Number of data points in the train set: 200, number of used features: 25
[LightGBM] [Info] Start training from score -1.021651
[LightGBM] [Info] Start training from score -1.187444
[LightGBM] [Info] Start training from score -1.093625
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

In [71]:
def generate_text(emotion):
    prompt = {
        "Positive": "Write a cheerful passage about a happiness.",
        "Negative": "Create motivational text to overcome frustration and sadness.",
        "Neutral": "Compose a reflective and serene passage on mindfulness."
    }[emotion]
    generator = pipeline("text-generation", model="gpt2")
    text_output = generator(prompt, max_length=1000, num_return_sequences=1, truncation=True)
    poem = text_output[0]['generated_text']
    print(f"Generated text for {emotion}:\n{poem}")





In [72]:
def generate_music(emotion):
    prompt = {
        "Positive": "Generate an upbeat melody with cheerful vibes.",
        "Negative": "Create a strong and intense rock beat.",
        "Neutral": "Compose a calm and relaxing instrumental piece."
    }[emotion]

    # Use text-to-audio pipeline
    generator = pipeline("text-to-audio", model="facebook/musicgen-small")
    output = generator(prompt)
    with open(f"{emotion}_music.mid", "wb") as f:
        f.write(output["audio"])
    print(f"Generated {emotion} music saved as {emotion}_music.mid")





In [73]:
def process_single_file(test_dir, file_name):
    file_path = os.path.join(test_dir, file_name)
    if not os.path.isfile(file_path):
        print(f"Error: {file_name} does not exist in {test_dir}")
        return
    if not file_name.endswith(('.wav', '.mp3')):
        print(f"Error: {file_name} is not a valid audio file.")
        return
    features = extract_features(file_path)
    prediction = clf.predict([features])
    emotion = encoder.inverse_transform(prediction)
    # Generate music and text
    print(f"Processed file: {file_name} | Predicted emotion: {emotion[0]}")
    generate_music(emotion[0])
    generate_text(emotion[0])


In [74]:
test_dir = '/root/.cache/kagglehub/datasets/imsparsh/audio-speech-sentiment/versions/2/TEST'
file_name = input("Enter the file name (with extension) from the test directory: ")
process_single_file(test_dir, file_name)


Enter the file name (with extension) from the test directory: 4.wav
Processed file: 4.wav | Predicted emotion: Positive


/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 5

Generated Positive music saved as Positive_music.mid


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated text for Positive:
Write a cheerful passage about a happiness. The point is this: "And how in heaven can happiness be achieved if we're not so humble?" And then if you take away another question about happiness, such as "How is happiness good?" and "It's not important that you always worry about it," then I will never read someone who says something like "It's important that you always be ready to make good decisions to give up something or move on from something" and then say, "It's not so important that you always have an idea for what you want to do or be. It's not important that one moment you think about happiness is important. You just need to take up something else." And then the way I look at it, what matters is not happiness, but what's there to be excited about."

What is the point of life if you do not know what will happen in the future? This would probably be like one of those existential scenarios where you say, "No, it won't happen by accident but I'm glad at t